In [8]:
import os

CAR_PATH = os.path.join("datasets\cars", "car.txt")

In [9]:
import pandas as pd 
cars = pd.read_csv(CAR_PATH, delimiter = ",", header=None)
cars.columns = ["Buying", "Maint", "Doors", "Persons", "Lug_boot", "Safety", "Acceptability"]
cars

,Buying,Maint,Doors,Persons,Lug_boot,Safety,Acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [10]:
for col in cars.columns:
    print(cars[col].value_counts())

vhigh    432
high     432
low      432
med      432
Name: Buying, dtype: int64
vhigh    432
high     432
low      432
med      432
Name: Maint, dtype: int64
2        432
5more    432
3        432
4        432
Name: Doors, dtype: int64
2       576
more    576
4       576
Name: Persons, dtype: int64
small    576
big      576
med      576
Name: Lug_boot, dtype: int64
high    576
low     576
med     576
Name: Safety, dtype: int64
unacc    1210
acc       384
good       69
vgood      65
Name: Acceptability, dtype: int64


In [11]:
# We have to stratify Acceptability since it's skewed
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(cars, cars[["Acceptability"]]): # Using StratifiedShuffleSplit and splitting using housing["income_cat"], we ensured that the proportion of each category of housing["income_cat"] is the same in the training and test sets
    strat_train_full_set = cars.loc[train_index]
    strat_test_set = cars.loc[test_index]
for train_index, valid_index in split.split(strat_train_full_set, strat_train_full_set[["Acceptability"]]):
    strat_train_set = strat_train_full_set.iloc[train_index] # loc does not work
    strat_valid_set = strat_train_full_set.iloc[valid_index]

In [12]:
strat_train_set["Acceptability"].value_counts() / len(strat_train_set)

unacc    0.700452
acc      0.221719
good     0.039819
vgood    0.038009
Name: Acceptability, dtype: float64

In [13]:
strat_valid_set["Acceptability"].value_counts() / len(strat_valid_set)

unacc    0.700361
acc      0.223827
good     0.039711
vgood    0.036101
Name: Acceptability, dtype: float64

In [14]:
strat_test_set["Acceptability"].value_counts() / len(strat_test_set)

unacc    0.699422
acc      0.222543
good     0.040462
vgood    0.037572
Name: Acceptability, dtype: float64

In [15]:
strat_train_set_data = strat_train_set.drop("Acceptability", axis=1) # drop labels for training set
strat_train_set_labels = strat_train_set["Acceptability"].copy()
strat_valid_set_data = strat_valid_set.drop("Acceptability", axis=1) # drop labels for training set
strat_valid_set_labels = strat_valid_set["Acceptability"].copy()

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

data_cat_attribs = ["Buying", "Maint", "Doors", "Persons", "Lug_boot", "Safety"]
data_pipeline = ColumnTransformer([
        ("oHCat", OneHotEncoder(), data_cat_attribs),
])
strat_train_set_data_encoded = data_pipeline.fit_transform(strat_train_set_data)
strat_valid_set_data_encoded = data_pipeline.fit_transform(strat_valid_set_data)

In [17]:
strat_train_set_data_encoded

<1105x21 sparse matrix of type '<class 'numpy.float64'>'
	with 6630 stored elements in Compressed Sparse Row format>

In [18]:
import numpy as np
strat_train_set_labels = np.array(strat_train_set_labels.values.tolist())
strat_train_set_labels = strat_train_set_labels.reshape(-1, 1) # Need to be a 1D matrix, not a 1D vector, for ordinal_encoder
strat_valid_set_labels = np.array(strat_valid_set_labels.values.tolist())
strat_valid_set_labels = strat_valid_set_labels.reshape(-1, 1)

In [19]:
strat_valid_set_labels.shape # Shape should be (277,1) not (277,)

(277, 1)

In [20]:
ordinal_encoder = OrdinalEncoder() # Instead of one-hot, we use ordinal. This is because the magnitude of the labels indicate some level difference
strat_train_set_labels_encoded = ordinal_encoder.fit_transform(strat_train_set_labels) # Change different text categories to numbers
strat_valid_set_labels_encoded = ordinal_encoder.fit_transform(strat_valid_set_labels)
strat_train_set_labels_encoded[:10]

array([[2.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [2.],
       [0.]])

In [21]:
print(np.unique(strat_train_set_labels_encoded))
print(np.unique(strat_valid_set_labels_encoded))
print(strat_train_set_labels_encoded.shape)
print(strat_valid_set_labels_encoded.shape)

[0. 1. 2. 3.]
[0. 1. 2. 3.]
(1105, 1)
(277, 1)


In [22]:
strat_train_set_labels_encoded = strat_train_set_labels_encoded.flatten() # Some algorithms expect a 1D vector not a 1D matrix
strat_valid_set_labels_encoded = strat_valid_set_labels_encoded.flatten()

In [23]:
print(np.unique(strat_train_set_labels_encoded))
print(np.unique(strat_valid_set_labels_encoded))
print(strat_train_set_labels_encoded.shape)
print(strat_valid_set_labels_encoded.shape)

[0. 1. 2. 3.]
[0. 1. 2. 3.]
(1105,)
(277,)


In [24]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(max_iter = 5000, random_state=42)
sgd_clf.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=5000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(sgd_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test indices such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.8590785907859079
0.8590785907859079
0.8555858310626703


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = [
    # try 9 (3×3) combinations of hyperparameters
    {'n_estimators': [10,30,100], 'max_features': [8,10,12]},
    # then try 9 (9×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [10,30,100], 'max_features': [8,10,12]},
]
forest_clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(forest_clf, param_grid, cv=5, scoring='f1_weighted', return_train_score=True)
grid_search.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{

In [27]:
for_clf = grid_search.best_estimator_
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(for_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.943089430894309
0.9349593495934959
0.9509536784741145


In [28]:
print(grid_search.best_params_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

{'max_features': 12, 'n_estimators': 30}
0.929691182772363 {'max_features': 8, 'n_estimators': 10}
0.9403809751274504 {'max_features': 8, 'n_estimators': 30}
0.9500815521965411 {'max_features': 8, 'n_estimators': 100}
0.9293743861022711 {'max_features': 10, 'n_estimators': 10}
0.9457516845255892 {'max_features': 10, 'n_estimators': 30}
0.9485008875115397 {'max_features': 10, 'n_estimators': 100}
0.9455154048776516 {'max_features': 12, 'n_estimators': 10}
0.9567003357810132 {'max_features': 12, 'n_estimators': 30}
0.9557733315721635 {'max_features': 12, 'n_estimators': 100}
0.9428576453627583 {'bootstrap': False, 'max_features': 8, 'n_estimators': 10}
0.9498782113510762 {'bootstrap': False, 'max_features': 8, 'n_estimators': 30}
0.9475272112434501 {'bootstrap': False, 'max_features': 8, 'n_estimators': 100}
0.945088272530604 {'bootstrap': False, 'max_features': 10, 'n_estimators': 10}
0.9512309058215088 {'bootstrap': False, 'max_features': 10, 'n_estimators': 30}
0.9557371218490074 {'bo

In [29]:
# Now let's try the above using randomizedsearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
}
forest_clf = RandomForestClassifier(random_state=42)
rnd_search = RandomizedSearchCV(forest_clf, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='f1_weighted', random_state=42)
rnd_search.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)
# The warnings are saying that some labels in y_true don't appear in y_pred

c:\users\taij\documents\calvin\myprojects\env\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\taij\documents\calvin\myprojects\env\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [30]:
for_clf = rnd_search.best_estimator_
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(for_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.9214092140921409
0.94579945799458
0.9427792915531336


In [31]:
print(rnd_search.best_params_)
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

{'max_features': 7, 'n_estimators': 48}
0.9422854273370739 {'max_features': 7, 'n_estimators': 180}
0.9312092226691626 {'max_features': 5, 'n_estimators': 15}
0.9203173415131247 {'max_features': 3, 'n_estimators': 72}
0.9269449296162904 {'max_features': 5, 'n_estimators': 21}
0.9423627571854498 {'max_features': 7, 'n_estimators': 122}
0.9242212728297717 {'max_features': 3, 'n_estimators': 75}
0.9217607183647717 {'max_features': 3, 'n_estimators': 88}
0.9453559493693303 {'max_features': 5, 'n_estimators': 100}
0.9206255817212562 {'max_features': 3, 'n_estimators': 150}
0.8300657246490976 {'max_features': 5, 'n_estimators': 2}
0.9284530728497802 {'max_features': 4, 'n_estimators': 158}
0.9421342448164106 {'max_features': 6, 'n_estimators': 130}
0.9243939933804599 {'max_features': 4, 'n_estimators': 21}
0.862881023184831 {'max_features': 1, 'n_estimators': 58}
0.9430050474629227 {'max_features': 6, 'n_estimators': 89}
0.8608553933929777 {'max_features': 1, 'n_estimators': 59}
0.9433849702

In [32]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(log_reg)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.8753387533875339
0.8644986449864499
0.8664850136239782


In [33]:
# Usually we would keep on tuning the paramters on the validation set, here I picked the model that performs the best and use it on the validation set
for_clf = RandomForestClassifier(n_estimators=30, max_features=12, random_state=42)
for_clf.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded) # Fits the training set, but predicts the valid set
y_pred = for_clf.predict(strat_valid_set_data_encoded)

In [34]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
print(confusion_matrix(strat_valid_set_labels_encoded, y_pred)) # arguments: actual, predict

[[ 60   0   1   1]
 [  1   9   0   1]
 [  1   0 193   0]
 [  0   0   0  10]]


In [35]:
print(classification_report(strat_valid_set_labels_encoded, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97        62
         1.0       1.00      0.82      0.90        11
         2.0       0.99      0.99      0.99       194
         3.0       0.83      1.00      0.91        10

    accuracy                           0.98       277
   macro avg       0.95      0.95      0.94       277
weighted avg       0.98      0.98      0.98       277



In [36]:
print(precision_score(strat_valid_set_labels_encoded, y_pred, average='weighted'))
print(recall_score(strat_valid_set_labels_encoded, y_pred, average='weighted'))
print(f1_score(strat_valid_set_labels_encoded, y_pred, average='weighted')) # the greater the precison & recall, the greater the f1_score and the better the classifier!

0.9831528279181708
0.9819494584837545
0.9819166393173612


In [38]:
import tensorflow as tf
from tensorflow import keras
# GPUs config
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus) # Looks like we have one GPU, nice!
# Config for GPU memory growth if needed
tf.config.experimental.set_memory_growth(gpus[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [39]:
def build_model(n_hidden=3, n_neurons=30, learning_rate=3e-3, input_shape=[21]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(4, activation="softmax")) # activation="sigmoid" for binary outputs
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(metrics= ['acc'], loss="sparse_categorical_crossentropy", optimizer=optimizer)
    return model

In [40]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [41]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-5, 3e-1),
}

# strat_train_set_data_encoded = strat_train_set_data_encoded.toarray()
# strat_train_set_labels_encoded = strat_train_set_labels_encoded.toarray()
# strat_valid_set_data_encoded = strat_valid_set_data_encoded.toarray()
# strat_valid_set_labels_encoded = strat_valid_set_labels_encoded.toarray()

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=50, cv=3, verbose=2)
rnd_search_cv.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded, epochs=100,
                  validation_data=(strat_valid_set_data_encoded, strat_valid_set_labels_encoded),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)])

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] learning_rate=0.06267826070237907, n_hidden=2, n_neurons=77 .....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 736 samples, validate on 277 samples
Epoch 1/100
736/736 [==============================] - 0s 619us/sample - loss: 0.9246 - acc: 0.6943 - val_loss: 0.7732 - val_acc: 0.7004
Epoch 2/100
736/736 [==============================] - 0s 79us/sample - loss: 0.7136 - acc: 0.7038 - val_loss: 0.6665 - val_acc: 0.7004
Epoch 3/100
736/736 [==============================] - 0s 79us/sample - loss: 0.6070 - acc: 0.7106 - val_loss: 0.5759 - val_acc: 0.7437
Epoch 4/100
736/736 [==============================] - 0s 72us/sample - loss: 0.5174 - acc: 0.7799 - val_loss: 0.5075 - val_acc: 0.7690
Epoch 5/100
736/736 [==============================] - 0s 73us/sample - loss: 0.4497 - acc: 0.8288 - val_loss: 0.4619 - val_acc: 0.7834
Epoch 6/100
736/736 [==============================] - 0s 73us/sample - loss: 0.4032 - acc: 0.8492 - val_loss: 0.4090 - val_acc: 0.8448
Epoch 7/100
736/736 [==============================] - 0s 73us/sample - loss: 0.3586 - acc: 0.8736 - val_loss: 0.3782 - val_acc: 0.8448
E

736/736 [==============================] - 0s 73us/sample - loss: 0.0348 - acc: 0.9986 - val_loss: 0.0941 - val_acc: 0.9675
Epoch 60/100
736/736 [==============================] - 0s 70us/sample - loss: 0.0340 - acc: 1.0000 - val_loss: 0.1030 - val_acc: 0.9603
Epoch 61/100
736/736 [==============================] - 0s 70us/sample - loss: 0.0337 - acc: 1.0000 - val_loss: 0.0975 - val_acc: 0.9603
Epoch 62/100
736/736 [==============================] - 0s 72us/sample - loss: 0.0320 - acc: 0.9986 - val_loss: 0.0933 - val_acc: 0.9639
Epoch 63/100
736/736 [==============================] - 0s 70us/sample - loss: 0.0310 - acc: 1.0000 - val_loss: 0.0995 - val_acc: 0.9603
Epoch 64/100
736/736 [==============================] - 0s 69us/sample - loss: 0.0302 - acc: 1.0000 - val_loss: 0.1022 - val_acc: 0.9531
Epoch 65/100
736/736 [==============================] - 0s 68us/sample - loss: 0.0294 - acc: 1.0000 - val_loss: 0.0934 - val_acc: 0.9603
Epoch 66/100
736/736 [==============================] 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.7s remaining:    0.0s


...

Epoch 86/100
737/737 [==============================] - 0s 68us/sample - loss: 0.4152 - acc: 0.8684 - val_loss: 0.4502 - val_acc: 0.8051
Epoch 87/100
737/737 [==============================] - 0s 83us/sample - loss: 0.4133 - acc: 0.8657 - val_loss: 0.4485 - val_acc: 0.7978
Epoch 88/100
737/737 [==============================] - 0s 70us/sample - loss: 0.4113 - acc: 0.8616 - val_loss: 0.4468 - val_acc: 0.8014
Epoch 89/100
737/737 [==============================] - 0s 69us/sample - loss: 0.4095 - acc: 0.8657 - val_loss: 0.4450 - val_acc: 0.8087
Epoch 90/100
737/737 [==============================] - 0s 66us/sample - loss: 0.4075 - acc: 0.8711 - val_loss: 0.4433 - val_acc: 0.8087
Epoch 91/100
737/737 [==============================] - 0s 66us/sample - loss: 0.4056 - acc: 0.8684 - val_loss: 0.4416 - val_acc: 0.8087
Epoch 92/100
737/737 [==============================] - 0s 68us/sample - loss: 0.4038 - acc: 0.8684 - val_loss: 0.4400 - val_acc: 0.8087
Epoch 93/100
737/737 [===================

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 14.0min finished
c:\users\taij\documents\calvin\myprojects\env\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


1105/1105 [==============================] - 0s 147us/sample - loss: 0.7428 - acc: 0.7158 - val_loss: 0.5483 - val_acc: 0.7545
Epoch 2/100
1105/1105 [==============================] - 0s 73us/sample - loss: 0.4391 - acc: 0.8308 - val_loss: 0.4179 - val_acc: 0.8087
Epoch 3/100
1105/1105 [==============================] - 0s 67us/sample - loss: 0.3369 - acc: 0.8670 - val_loss: 0.3839 - val_acc: 0.8087
Epoch 4/100
1105/1105 [==============================] - 0s 65us/sample - loss: 0.2811 - acc: 0.8824 - val_loss: 0.2718 - val_acc: 0.8845
Epoch 5/100
1105/1105 [==============================] - 0s 66us/sample - loss: 0.2367 - acc: 0.9023 - val_loss: 0.2764 - val_acc: 0.8953
Epoch 6/100
1105/1105 [==============================] - 0s 67us/sample - loss: 0.2047 - acc: 0.9276 - val_loss: 0.2197 - val_acc: 0.9134
Epoch 7/100
1105/1105 [==============================] - 0s 64us/sample - loss: 0.1858 - acc: 0.9321 - val_loss: 0.1962 - val_acc: 0.9422
Epoch 8/100
1105/1105 [======================

1105/1105 [==============================] - 0s 60us/sample - loss: 0.0064 - acc: 1.0000 - val_loss: 0.0302 - val_acc: 0.9856
Epoch 61/100
1105/1105 [==============================] - 0s 64us/sample - loss: 0.0062 - acc: 1.0000 - val_loss: 0.0309 - val_acc: 0.9856
Epoch 62/100
1105/1105 [==============================] - 0s 63us/sample - loss: 0.0061 - acc: 1.0000 - val_loss: 0.0319 - val_acc: 0.9856
Epoch 63/100
1105/1105 [==============================] - 0s 64us/sample - loss: 0.0058 - acc: 1.0000 - val_loss: 0.0285 - val_acc: 0.9856
Epoch 64/100
1105/1105 [==============================] - 0s 64us/sample - loss: 0.0057 - acc: 1.0000 - val_loss: 0.0283 - val_acc: 0.9856
Epoch 65/100
1105/1105 [==============================] - 0s 63us/sample - loss: 0.0055 - acc: 1.0000 - val_loss: 0.0290 - val_acc: 0.9856
Epoch 66/100
1105/1105 [==============================] - 0s 65us/sample - loss: 0.0053 - acc: 1.0000 - val_loss: 0.0285 - val_acc: 0.9856
Epoch 67/100
1105/1105 [================

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000002CAB445E0C8>,
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002CAB4477A08>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score

In [49]:
rnd_search_cv.best_params_

{'learning_rate': 0.11238716908520452, 'n_hidden': 2, 'n_neurons': 88}

In [50]:
rnd_search_cv.best_score_ # score is the higher (more positive) the better

0.9710407138949606

In [51]:
cvres = rnd_search_cv.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]): # mean score is the higher (more positive) the better, since this is sklearn's RandomizedSearchCV
    print(mean_score, params)

0.9628959183779238 {'learning_rate': 0.06267826070237907, 'n_hidden': 2, 'n_neurons': 77}
0.6461538501994103 {'learning_rate': 0.00011677469047281985, 'n_hidden': 2, 'n_neurons': 51}
0.7574660700370823 {'learning_rate': 0.0020575898550161393, 'n_hidden': 1, 'n_neurons': 24}
0.5972850677654215 {'learning_rate': 9.68757463874861e-05, 'n_hidden': 3, 'n_neurons': 22}
0.9185520352821005 {'learning_rate': 0.009279322020780675, 'n_hidden': 3, 'n_neurons': 90}
0.7004524907375353 {'learning_rate': 0.0006860878068127496, 'n_hidden': 1, 'n_neurons': 44}
0.7900452449310958 {'learning_rate': 0.0030643122555674735, 'n_hidden': 1, 'n_neurons': 31}
0.7276018154567183 {'learning_rate': 0.0015305577001422787, 'n_hidden': 2, 'n_neurons': 42}
0.7999999957926133 {'learning_rate': 0.006385679874384236, 'n_hidden': 0, 'n_neurons': 90}
0.6018099449069252 {'learning_rate': 6.479590738411322e-05, 'n_hidden': 3, 'n_neurons': 58}
0.9665158399629377 {'learning_rate': 0.22605780263757855, 'n_hidden': 2, 'n_neurons'

In [52]:
print(rnd_search_cv.best_estimator_)
ANNmodel = rnd_search_cv.best_estimator_.model

In [53]:
y_pred = ANNmodel.predict_classes(strat_valid_set_data_encoded)

In [54]:
print(confusion_matrix(strat_valid_set_labels_encoded, y_pred)) # arguments: actual, predict

[[ 61   0   1   0]
 [  1   9   0   1]
 [  1   0 193   0]
 [  0   0   0  10]]


In [55]:
print(classification_report(strat_valid_set_labels_encoded, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98        62
         1.0       1.00      0.82      0.90        11
         2.0       0.99      0.99      0.99       194
         3.0       0.91      1.00      0.95        10

    accuracy                           0.99       277
   macro avg       0.97      0.95      0.96       277
weighted avg       0.99      0.99      0.99       277



In [78]:
final_model = ANNmodel
X_test = strat_test_set.drop("Acceptability", axis=1) 
X_test_prepared = data_pipeline.transform(X_test) # we do not fit (only transform) here because we want to use the fitted values of the training set to predict this, i.e., use the training set's median values and extra attributes to transform the test set
X_test_prepared = X_test_prepared.toarray()
y_test = strat_test_set["Acceptability"].copy()
y_test = np.array(y_test.values.tolist())
y_test = y_test.reshape(-1, 1)
y_test = ordinal_encoder.transform(y_test)
y_test = y_test.flatten()
final_predictions = final_model.predict_classes(X_test_prepared)

In [79]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
print(precision_score(y_test, final_predictions, average='weighted'))
print(recall_score(y_test, final_predictions, average='weighted'))
print(f1_score(y_test, final_predictions, average='weighted'))
print(confusion_matrix(y_test, final_predictions))

0.9971217203073338
0.9971098265895953
0.9971033611386672
[[ 76   0   1   0]
 [  0  14   0   0]
 [  0   0 242   0]
 [  0   0   0  13]]


In [80]:
y_pred = final_model.predict_classes(X_test_prepared)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99        77
         1.0       1.00      1.00      1.00        14
         2.0       1.00      1.00      1.00       242
         3.0       1.00      1.00      1.00        13

    accuracy                           1.00       346
   macro avg       1.00      1.00      1.00       346
weighted avg       1.00      1.00      1.00       346

